# Supervised Fine-Tuning

Supervised Fine-Tuning (SFT) is a process primarily used to adapt pre-trained language models to follow instructions, engage in dialogue, and use specific output formats. This is typically done by training on datasets of human-written conversations and instructions.


## When to Use SFT

As a first step, you should consider whether using an existing instruction-tuned model with well-crafted prompts would suffice for your use case. SFT involves significant computational resources and engineering effort, so it should only be pursued when prompting existing models proves insufficient.

> Consider SFT only if you: - Need additional performance beyond what prompting can achieve - Have a specific use case where the cost of using a large general-purpose model outweighs the cost of fine-tuning a smaller model - Require specialized output formats or domain-specific knowledge that existing models struggle with

### Template Control

SFT allows precise control over the model’s output structure. This is particularly valuable when you need the model to:

- Generate responses in a specific chat template format
- Follow strict output schemas
- Maintain consistent styling across responses

### Domain Adaptation

When working in specialized domains, SFT helps align the model with domain-specific requirements by:

- Teaching domain terminology and concepts
- Enforcing professional standards
- Handling technical queries appropriately
- Following industry-specific guidelines

> Before starting SFT, evaluate whether your use case requires: - Precise output formatting - Domain-specific knowledge - Consistent response patterns - Adherence to specific guidelines. This evaluation will help determine if SFT is the right approach for your needs.


## Dataset Preparation

The supervised fine-tuning process **requires a task-specific dataset** structured with input-output pairs. Each pair should consist of:

- An input prompt
- The expected model response
- Any additional context or metadata

The **quality of your training data is crucial** for successful fine-tuning. Let’s look at how to prepare and validate your dataset:


## Training Configuration

The success of your fine-tuning depends heavily on choosing the right training parameters. Let’s explore each important parameter and how to configure them effectively:

The SFTTrainer configuration requires consideration of several parameters that control the training process. Let’s explore each parameter and their purpose:

1. **Training Duration Parameters**:
  - `num_train_epochs`: Controls total training duration
  - `max_steps`: Alternative to epochs, sets maximum number of training steps
  - More epochs allow better learning but risk overfitting

2. **Batch Size Parameters**:
  - `per_device_train_batch_size`: Determines memory usage and training stability
  - `gradient_accumulation_steps`: Enables larger effective batch sizes
  - Larger batches provide more stable gradients but require more memory


3. **Learning Rate Parameters**:
  - `learning_rate`: Controls size of weight updates
  - `warmup_ratio`: Portion of training used for learning rate warmup
  - Too high can cause instability, too low results in slow learning

4. **Monitoring Parameters**:
  - `logging_steps`: Frequency of metric logging
  - `eval_steps`: How often to evaluate on validation data
  - `save_steps`: Frequency of model checkpoint saves

> Start with conservative values and adjust based on monitoring: - Begin with 1-3 epochs - Use smaller batch sizes initially - Monitor validation metrics closely - Adjust learning rate if training is unstable


## Implementation with TRL

Now that we understand the key components, let’s implement the training with proper validation and monitoring. We will use the `SFTTrainer` class from the Transformers Reinforcement Learning (TRL) library, which is built on top of the `transformers` library. Here’s a complete example using the TRL library:

In [ ]:
import torch
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Set device
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

print(f"Using device: {device}")

### Load the model

In [ ]:
# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

[Set up the chat format](https://huggingface.co/docs/trl/sft_trainer#add-special-tokens-for-chat-format). The `setup_chat_format()` function in trl easily sets up a model and tokenizer for conversational AI tasks. This function:

- Adds special tokens to the tokenizer, e.g. `<|im_start|>` and `<|im_end|>`, to indicate the start and end of a conversation.
- Resizes the model’s embedding layer to accommodate the new tokens.
- Sets the `chat_template` of the tokenizer, which is used to format the input data into a chat-like format. The default is chatml from OpenAI.

In [ ]:
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved
finetune_name = "SmolLM2-FT-MyDataset"

### Load the dataset

In [ ]:
# Load dataset
dataset = load_dataset(
    path="HuggingFaceTB/smoltalk", name="everyday-conversations"
)
dataset

In [ ]:
# print first example
print(dataset["train"][0])

### Generate with the base model

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Finetune the model

[Daset format support](https://huggingface.co/docs/trl/sft_trainer#dataset-format-support). The `SFTTrainer` supports popular dataset formats. This allows you to pass the dataset to the trainer without any pre-processing directly. The following formats are supported:

- conversational format
```python
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "Who wrote 'Romeo and Juliet'?"}, {"role": "assistant", "content": "..."}]}
{"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "How far is the Moon from Earth?"}, {"role": "assistant", "content": "..."}]}
```

- instruction format
```python
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

If your dataset uses one of the above formats, you can **directly pass it to the trainer without pre-processing**. The SFTTrainer will then format the dataset for you using the defined format from the model’s tokenizer with the apply_chat_template method.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
)

In [ ]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    tokenizer=tokenizer,
    eval_dataset=dataset["test"],
)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Save the model
trainer.save_model(finetune_name)